In [ ]:
%matplotlib notebook

# This examples shows who to download files from the ONC server
import glob
import os

import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import strawb
import strawb.sensors.lidar
import strawb.tools

import plotly
import plotly.graph_objs as go
import plotly.express as px
import pandas
import plotly.io as pio
pio.templates.default = "plotly_white"

import random

plt.rcParams.update({"text.usetex": False})  # fix time fmt of x_ticks

# Load ONC DB and mask files of interest

In [ ]:
# Check if DB exits, if not load it, but update it anyway
if os.path.exists(strawb.Config.pandas_file_sync_db):
    db = strawb.SyncDBHandler()  # loads the db from disc
else:
    db = strawb.SyncDBHandler(load_db=False)  # loads the db from ONC server

db.load_onc_db_update(output=True, save_db=True)

In [ ]:
# mask by device
mask = db.dataframe['deviceCode'] == 'TUMLIDAR001'
mask |= db.dataframe['deviceCode'] == 'TUMLIDAR002'

mask &= db.dataframe.dataProductCode == 'LIDARSD'

# 'measurement_type', 'file_version' and others are available once the files are synced
# as the ONC DB isn't holding those informations. 
# A workaround is to donwload the DB from the VM which has all files loaded.
# mask &= db.dataframe['measurement_type'] == 'laser_adjustment'

db.dataframe[mask]  # plot the table

# Get one file by time

In [ ]:
mask_f = mask.copy()

# get one specified file - 'TUMLIDAR001_20211118T153858.552Z-SDAQ-LIDAR.hdf5'
mask_f &= db.dataframe.dateFrom >= "2021-11-18T1530"
mask_f &= db.dataframe.dateFrom < "2021-11-18T1540"

db.dataframe[mask_f]  # plot the table

In [ ]:
# download the file, in case it isn't synced - otherwise skipp it
db.get_files_from_names(db.dataframe[mask_f].filename)

## Load the file

In [ ]:
lidar = strawb.sensors.lidar.Lidar(file=db.dataframe[mask_f].fullPath.iloc[0])
lidar.file_handler.file_attributes  # show the file attributes, time in seconds since epoch (1.1.1970) UTC

In [ ]:
# What is in the file:
for i in lidar.file_handler.__members__:
    print(f'{i:30} : {lidar.file_handler.__getattribute__(i)}')

# Plotting with pandas and plotly

## Set up plotting function

In [ ]:
def multiline_plot(data_frame):
    columns = list(set(data_frame.columns).difference(['time']))
    
    fig = plotly.subplots.make_subplots(shared_xaxes=True,
                                        rows=len(columns),
                                        cols=1, 
                                        #subplot_titles=columns
                                       )


    fig.update_layout(
        width=800,
        height=150+100*len(columns),)

    j = 1
    for i, column_i in enumerate(columns):
        fig.add_trace(go.Scatter(x=data_frame['time'], 
                                 y=data_frame[column_i],
                                 name=column_i), 
                      row=i+1, col=1
                     )

    return fig

## Get the DataFrames

In [ ]:
df_daq = lidar.file_handler.get_pandas_daq()
df_gimbal = lidar.file_handler.get_pandas_gimbal()
df_laser = lidar.file_handler.get_pandas_laser()

df_tot = lidar.file_handler.get_pandas_tot()
df_counts = lidar.file_handler.get_pandas_counts()

df_measurement = lidar.file_handler.get_pandas_measurement()

df_dcounts = lidar.trb_rates.get_pandas_dcounts()
df_rates = lidar.trb_rates.get_pandas_rate()

## Basic plots to show the dataFrames

In [ ]:
fig = multiline_plot(df_daq)
fig.show()

In [ ]:
fig = multiline_plot(df_gimbal)
fig.show()

In [ ]:
fig = multiline_plot(df_laser)
fig.show()

In [ ]:
fig = multiline_plot(df_measurement)
fig.show()

In [ ]:
# Plot the first 10'000 TOT data, as all is a lot of data
fig = multiline_plot(df_tot.iloc[:10000])
fig.show()

df_tot.shape

In [ ]:
# Plot the first 10'000 counter data, as it's less but 100Hz has some size.
# Moreover those values have to be processed further. -> df_rates
fig = multiline_plot(df_counts.iloc[:10000])
fig.show()

df_counts.shape

# Counts and Rates

In [ ]:
# dcounts are the diff of the stored TRB counts readings. As the readings can jitter a bit, dcounts_pmt jitter, too
# to correct devide by dcounts_time to get a rate (or take df_rates) or 
# by dcounts_laser to get a success rate [events/trigger]
fig = multiline_plot(df_dcounts.iloc[:10000])
fig.show()

In [ ]:
fig = multiline_plot(df_rates.iloc[:10000])
fig.show()